##Imports

In [ ]:
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import nltk
nltk.download('punkt')
nltk.download('stopwords')

##Load data

In [ ]:
df = pd.read_excel('/content/sample_data/candidates_resume.xlsx')

In [ ]:
df.info()

NameError: ignored

In [ ]:
df.head(10)

,Nome_Candidato,Perfil,Curso,Formacao_Academica,Certificado,Experiencia,Descricao_Vaga,Status_Selecao,Etapa_Selecao,Palavra_Chave,Indicacao_projeto,Novas palavras-chave
0,CD - 00001,tua há mais de 6 anos com desenvolvimento de J...,,Graduação em Análise e Desenvolvimento de Sist...,Oracle Cloud Infrastructure Foundations 2021 A...,"Arquiteto Java / Tech Lead, Compre & Alugue Ag...",Arquiteto de Software Sr,Em análise,Disponível,"EJB, JSE,JEE,SOA, SQL, UML, XML, WSDL, XSD, XP...",MAPA,
1,CD - 00002,Profissional se reconhece como sendo Líder de ...,,Centro Universitário das Faculdades Metropolit...,Six Sigma Yellow Belt pela 6SigmaStudy\n-Custo...,Fivetech Tecnologia Aplicada Ltda - Coordenado...,Coordenador de Operações,Em análise,Disponível,• Gestão de equipe • Habilidades em comunicaçã...,Bradesco,
2,CD - 00003,"Trabalha há mais de 3 anos com PHP, desenvol...",,Graduação em Rede de Computadores - Concluída ...,,Meta\nDesenvolvedor web PHP Pleno\nmaio de 202...,Desenvolvedor PHP Pleno,Em análise,Disponível,"PHP, Zend, Docker, SOLID, TDD",MAPA,
3,CD - 00004,Especialista em MAINFRAME como Analista de Sis...,Ano Formação / Certificação\n1994 Pro...,FORMAÇÃO ACADÉMICA\nCURSO Administração de emp...,Não possui.,UNIVERSIDADE ESTÁCIO DE SÁ -Analista de Teste...,Analista de Requisitos,Disponível,Disponível,"Analista de Sistemas, Testes, Requisitos, Dev.",,
4,CD - 00005,atua há mais de 3 anos com desenvolvimento de ...,,Graduação em Sistemas da Informação - Concluíd...,"Não tem, mas está disposto a tirar e está cien...",Outubro de 2021a Dezembro de 2022 - Programado...,Desenvolvedor Java Pleno,Contratado,Disponível,,MAPA,
5,CD - 00006,"Profissional possui experiência em: HTML, Css,...",,"GRADUADO, Faculdade: UNIPÊ - CENTRO UNIVERSITÁ...",Ruby on Rails 5 parte 1: Fundamentos\nEntity F...,Tipo: TRABALHO\nEmpresa: INDRA/MINSAIT\nInício...,Analista Desenvolvedor Salesforce,Em análise,Disponível,"Mobilte programming, Web Designer (Photoshop, ...",Claro salerforce,
6,CD - 00007,"Sou Analista Desenvolvedor Java, profissional ...",,Grau de Formação: Bacharel em Sistema de Infor...,(Empresa BSI Tecnologia) Certificado do Curso ...,Cartões Corporativos - Atuei no projeto de car...,Desenvolvedor Java.,Contratado,Disponível,"Struts, Javam Jenkins, Docker",MAPA,
7,CD - 00008,Experiência como Gerente de Projetos de TI na ...,Preparatório para certificação PMI-ACP\nPROJET...,Grau de Formação: Pós-Graduação “Stricto Sensu...,• Certified ScrumMaster® (CSM) - Scrum Allianc...,BRQ Digital Solutions (Empresa Atual)\nCargo.....,PRODUCT OWNER SALESFORCE,Em análise,Disponível,": C#, MS-Visual Studio IDE, Borland Delphi, Pa...",Claro,
8,CD - 00009,Profissional com sólida experiência em análise...,Scrum Foundation Professional Certificate - Ce...,Tecnólogo em Análise e Desenvolvimento de Sist...,Não tem mas está disposto a tirar e está cient...,Sinqia S.A.\n18/09/2019 - 15/12/2019 - Estágia...,Desenvolvedor NodeJS Pleno,Em análise,Disponível,"r FileMaker Database, SLQServer e MySQL.",MAPA,
9,CD - 00010,Profissional com mais de 8 anos de experencia ...,,-UFRJ - Universidade Federal do Rio de Janeiro...,,CASNAV - Centro de Análises de Sistemas Navais...,Analista de Requisitos Sênior,Em análise,Disponível,Planejamento de testes Testes manuais Teste de...,INCA,


##Data Preparation


In [ ]:
df = df.fillna('')

In [ ]:
df['Perfil'] = df['Perfil'].astype(str)
df['Curso'] = df['Curso'].astype(str)
df['Formacao_Academica'] = df['Formacao_Academica'].astype(str)
df['Certificado'] = df['Certificado'].astype(str)
df['Experiencia'] = df['Experiencia'].astype(str)

In [ ]:
def custom_tokenize(text):
    # Set pattern to preserve expressions with stop signs
    pattern = r"\w+(?:\.\w+)+|\w+"

    # Tokenize text using custom pattern
    tokens = re.findall(pattern, text)

    return tokens

In [ ]:

def extract_keywords_tfidf(text, num_keywords):
  try:
    if len(text) < 20:
      return
    # words Tokenization
    #tokens = word_tokenize(text)
    tokens = custom_tokenize(text)

    # Stopwords removal
    stop_words = set(stopwords.words('portuguese'))
    filtered_tokens = [token for token in tokens if token.lower() not in stop_words]

    # Concatenate tokens into a single string
    processed_text = ' '.join(filtered_tokens)

    # Create a TF-IDF vectorizer
    vectorizer = TfidfVectorizer(max_features=num_keywords)

    # Apply vectorizer to processed text
    X = vectorizer.fit_transform([processed_text])

    # Get the keywords with the highest TF-IDF weights
    feature_names = vectorizer.get_feature_names_out()
    keyword_indices = X.toarray().argsort()[0][-num_keywords:][::-1]
    keywords = [feature_names[idx] for idx in keyword_indices]
  except:
    return
  else:
    return keywords


##Keywords Extraction

In [ ]:
# Set the number of desired keywords
num_keywords = 15

# Extract keywords using TF-IDF algorithm
df['Perfil_keywords'] = df.apply(lambda row: extract_keywords_tfidf(row['Perfil'], num_keywords), axis=1)
df['Curso_keywords'] = df.apply(lambda row: extract_keywords_tfidf(row['Curso'], num_keywords), axis=1)
df['Formacao_keywords'] = df.apply(lambda row: extract_keywords_tfidf(row['Formacao_Academica'], num_keywords), axis=1)
df['Certificado_keywords'] = df.apply(lambda row: extract_keywords_tfidf(row['Certificado'], num_keywords), axis=1)
df['Experiencia_keywords'] = df.apply(lambda row: extract_keywords_tfidf(row['Experiencia'], num_keywords), axis=1)



In [ ]:
#Export new df (will be used in Power BI)
df.to_excel('/content/sample_data/TF_candidatos_trilha.xlsx', index=False)